# Data ETL using billboard and spotify data


In [ ]:
# all packages
import billboard
import pandas as pd
import os

from oauthlib.oauth2 import BackendApplicationClient
from requests_oauthlib import OAuth2Session
import time,json
import pandas as pd
import requests
import re
import random
import urllib
import os.path

*****
## Step 1: collect billboard data
Songs date as far back as 1958, where I found the first available chart for Hot 100.

In [1]:
# pip install billboard.py

# in this part we want to build up a data frame of all available charts on billboard

if not os.path.isfile('Billboard_data'):
    Hot_10 = pd.DataFrame(columns = ['date','title','artist','rank','weeks','change_in_rank','spotifyID'
        ])


    chart = billboard.ChartData('hot-100',date = '2010-12-25') # this breaks sometimes so have to rerun and change date
    date = chart.date
    prev_year,prev_month = date.split('-')[0],date.split('-')[1]

    while chart.previousDate:
        year,month = date.split('-')[0],date.split('-')[1]
        if prev_month != month:
            print 'chart year:{},month:{}'.format(year,month)
        for i in range(10):
            song = chart[i]
            Hot_10 = Hot_10.append([(date,song.title,song.artist,song.rank,song.weeks,song.change,song.spotifyID)])
        chart = billboard.ChartData('hot-100',chart.previousDate)
        prev_year,prev_month = year,month
        date = chart.date
        no += 1


In [2]:
if not os.path.isfile('Billboard_data'):
    # the scrapping breaks sometimes due to website control so have to rerun and change date
    chart = billboard.ChartData('hot-100',date = '1958-08-09') 
    date = chart.date
    prev_year,prev_month = date.split('-')[0],date.split('-')[1]

    while chart.previousDate:
        year,month = date.split('-')[0],date.split('-')[1]
        if prev_month != month:
            print 'chart year:{},month:{}'.format(year,month)
        for i in range(10):
            song = chart[i]
            Hot_10 = Hot_10.append([(date,song.title,song.artist,song.rank,song.weeks,song.change,song.spotifyID)])
        chart = billboard.ChartData('hot-100',chart.previousDate)
        prev_year,prev_month = year,month
        date = chart.date
        no += 1
        
    Hot_10_dropped = Hot_10.drop(Hot_10.columns.values[7:],axis=1)
    Hot_10_dropped.columns = ['date','title','artist','rank','weeks','change_in_rank','spotifyID']
    Hot_10_dropped.to_pickle('Billboard_data')

In [3]:

top10 = pd.read_pickle('Billboard_data')
top10_spotify = top10[top10.spotifyID != u'']
top10_spotify.head(2)

,date,title,artist,rank,weeks,change_in_rank,spotifyID
0,2017-03-18,Shape Of You,Ed Sheeran,1.0,8.0,0,0FE9t6xYkqWXU2ahLh6D8X
0,2017-03-18,Bad And Boujee,Migos Featuring Lil Uzi Vert,2.0,16.0,0,1FvU97lrWOG2NRxErh6OZz


*****
## Step 2: collect spotify data

In [24]:
def get_token():
    auth_url = u'https://accounts.spotify.com/api/token'

    ##### private information #####
    client_id = u'client_id_string_here'
    client_secret = u'client_secret_string_here'
    ##### wipe if publish in public #####
    client = BackendApplicationClient(client_id=client_id)
    oauth = OAuth2Session(client=client)
    token = oauth.fetch_token(token_url=auth_url, client_id=client_id,
        client_secret=client_secret)

    return token

def get_spotify_json(url,token=None):
    if not token or token['expires_at']-time.time() < 100: # if token less than 100 seconds until expiration
        token = get_token()

    access_token = token['access_token']
    header = {u'Authorization':u'Bearer '+access_token}

    r = requests.get(url,headers=header)
    return r.json(),token

def single_audio_features(track_id,token=None):
    url_head = u'https://api.spotify.com/v1/audio-features/'

    if type(track_id) == str:
        url = url_head + track_id
    else:
        raise TypeError("Track_id should be str!")

    r,token = get_spotify_json(url,token)
    table = pd.DataFrame([r])
    table.drop(['analysis_url','id','track_href','type'],axis=1,inplace=True)
    return table,token
    
def multiple_audio_features(track_ids,token=None):
    url_head = u'https://api.spotify.com/v1/audio-features?ids='

    url = url_head + ','.join(track_ids)

    rs,token = get_spotify_json(url,token)
    table = pd.DataFrame()
    for r in rs['audio_features']:
        row = pd.DataFrame([r])
        row.drop(['analysis_url','id','track_href','type'],axis=1,inplace=True)
        table = table.append(row)

    return table,token

def chunk_100(series):
    for i in xrange(0,len(series),100):
        yield series[i:(i+100)]
    
def search_uri(title,artist,token=None):
    '''
    title eg.: 'never gonna give you up'
    '''
    url_head = u'https://api.spotify.com/v1/search?q='
    name = '+'.join(map(urllib.quote,unicode(title,errors='ignore').split(' ')))

    url = url_head+name+'&type=track&market=US&limit=20'
    
    rs,token = get_spotify_json(url,token)
    
    if 'tracks' in rs and 'items' in rs['tracks']:
        for result in rs['tracks']['items']:
            if result['album']['artists'][0]['name'].lower() == artist.lower():
                return result['id']
    return None


## Get audio features for billboard songs

In [9]:
top10_spotify = top10[top10.spotifyID != u'']
distinct_trackids = list(set(top10_spotify.spotifyID.values))

audio_data = pd.DataFrame()
if not os.path.isfile('Spotify_audio_features'):
    for i,track_ids in enumerate(chunk_100(distinct_trackids)):
        audio_features,token = multiple_audio_features(track_ids,token)
        audio_data = audio_data.append(audio_features)
        
    assert len(audio_data)==len(distinct_trackids)
    audio_data.to_pickle('Spotify_audio_features')

else:
    audio_data = pd.read_pickle('Spotify_audio_features')

## Get audio features for 4000 MSD songs

In [10]:
title_artist_spotify_set = set(zip(map(lambda x:x.lower(),top10_spotify.title.values),map(lambda x:x.lower(),top10_spotify.artist.values)))
print len(title_artist_spotify_set)

4314


In [19]:
def get_MSD_tracks(title_artist_set):
    MSD_tracks = []
    with open('./MSD/MillionSongSubset/AdditionalFiles/subset_unique_tracks.txt') as f:
        for i,line in enumerate(f):
            if len(MSD_tracks) % 100 == 0:
                print i,len(MSD_tracks)
            title,artist = re.split('<SEP>',line.replace('\n',''))[::-1][:2]
            pair = (title.lower(),artist.lower())

            if len(MSD_tracks) <= 4000:
                if pair not in title_artist_set:
                    uri = search_uri(title,artist)
                    if uri:
                        MSD_tracks.append((title,artist,uri))
            else:
                return MSD_tracks
        return MSD_tracks

if not os.path.isfile('MSD_tracks'):
    MSD_tracks = get_MSD_tracks(title_artist_spotify_set)
    MSD = pd.DataFrame(MSD_tracks,columns=['title','artist','uri'])
    MSD.to_pickle('MSD_tracks')
else:
    MSD = pd.read_pickle('MSD_tracks')
    MSD_track_ids = MSD.uri.values

In [21]:
MSD.head(2)

,title,artist,uri
0,I Didn't Mean To,Casual,01TR6aAKrA2cI3Z0gnCOsu
1,Soul Deep,The Box Tops,4yPl1mK1oluIrCwI4HInPR


In [13]:
audio_data = pd.DataFrame()
if not os.path.isfile('MSD_audio_features'):
    for i,track_ids in enumerate(chunk_100(MSD_track_ids)):
        audio_features,token = multiple_audio_features(track_ids)
        audio_data = audio_data.append(audio_features)
        
    assert len(audio_data)==len(MSD_track_ids)
    audio_data.to_pickle('MSD_audio_features')

else:
    audio_data = pd.read_pickle('MSD_audio_features')

*****
## Step 3 Get lyrics from lyrics.wikia.com

According to the original plan I was to get the lyrics from Musixmatch, but for their free version they only provide 30% of the lyrics per song, and has a quota of 2000 songs per day. So I looked for other resources and found this website, and scraped from its html code
link to the website:
http://lyrics.wikia.com/

Since it's a long process I singled the whole process out for a separate notebook. Please check '1.5 get lyrics.ipynb' for details

*****
## Step 4 Look for other songs by same billboard artists

From Step 3 I found that only 46% of the MSD songs have lyrics available, and from those whose lyrics are not available, I found that was because they were very unpopular songs, some in foreign languages even.(details can be found in '1.5 get lyrics.ipynb')

Therefore I decided to replace MSD songs with a list of songs by billboard artists, in that sense I can achieve two goals:
- 1 by limiting artists to those once have billboard hit songs, I can compare songs with their production companies a controlled variable, i.e. the influence of production companies will be limited and under control because of sourcing process of data.
- 2 I can still get 4000 more non-billboard songs, and get the lyrics of them and build the prediction model

In [41]:
def search_artist_id(artist,token=None):
    '''
    artist: single artist name, should be from get_spotify_names,i.e. spotify name for artist
    '''
    url_head = u'https://api.spotify.com/v1/search?q='
    name = '+'.join(map(urllib.quote,unicode(artist,errors='ignore').split(' ')))
    
    url = url_head+name+'&type=artist&market=US&limit=20'
    
    rs,token = get_spotify_json(url,token)
    
    if 'artists' in rs and 'items' in rs['artists']:
        for result in rs['artists']['items']:
            if result['name'].lower() == artist.lower():
                return result['id']
    return None

def get_artist_albums(artist_id,token=None):
    url_head = u'https://api.spotify.com/v1/artists/'
    
    url = url_head+artist_id+'/albums?market=US'
    
    rs,token = get_spotify_json(url,token)
    
    results = []
    if 'items' in rs:
        for result in rs['items']:
            r = result['name'],result['type'],result['uri'] #album,album_type,album_uri
            results.append(r)
        return results
    return None

def get_album_tracks(album_id,token=None):
    url_head = u'https://api.spotify.com/v1/albums/'
    
    url = url_head+album_id+'/tracks?market=US'
    
    rs,token = get_spotify_json(url,token)
    
    results = []
    if 'items' in rs:
        for result in rs['items']:
            r = result['name'],result['track_number'],result['uri'] #album,album_type,album_uri
            results.append(r)
        return results
    return None

In [104]:
test_artist_id = search_artist_id('michael jackson')

In [105]:
result = get_artist_albums(test_artist_id)

In [106]:
pd.DataFrame(result)

,0,1,2
0,XSCAPE,album,spotify:album:7pomP86PUhoJpY3fsC0WDQ
1,XSCAPE - Track by Track Commentary,album,spotify:album:54WFfZzds26NrwcmStfamd
2,Michael,album,spotify:album:0yYWeLqonFk11pmb0RiMbP
3,Invincible,album,spotify:album:52E4RP7XDzalpIrOgSTgiQ
4,BLOOD ON THE DANCE FLOOR/ HIStory In The Mix,album,spotify:album:0RNsFWWdiz1rrdLI1pwbvJ
5,"HIStory - PAST, PRESENT AND FUTURE - Book I",album,spotify:album:4Cq3OcsC1jw9fhHiVdsFXv
6,"HIStory - PAST, PRESENT AND FUTURE - BOOK I",album,spotify:album:3OBhnTLrvkoEEETjFA3Qfk
7,Dangerous,album,spotify:album:0oX4SealMgNXrvRDhqqOKg
8,Bad (Remastered),album,spotify:album:3Us57CjssWnHjTUIXBuIeH
9,Bad 25th Anniversary,album,spotify:album:24TAupSNVWSAHL0R7n71vm


Table:
| Artist | Artist ID | Album | Album Type | Track | Track Number | Track ID